## 트리의 앙상블

#### 정형 데이터 - 미리 정해진 구조에 따라 저장된 데이터 (표로 저장 가능)
#### 비정형 데이터 - 구조가 정해지지 않은 데이터

### 랜덤 포레스트 - 다수의 결정 트리 중 중복을 허용하여 데이터를 샘플링해 최종 예측

In [1]:
# 데이터 생성 및 분할
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine=pd.read_csv('https://bit.ly/wine_csv_data')
data=wine[['alcohol', 'sugar', 'pH']].to_numpy()
target=wine['class'].to_numpy()
train_input, test_input, train_target, test_target=train_test_split(data, target, test_size=0.2, random_state=42)


In [ ]:
# 교차 검증
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_jobs=-1, random_state=42)   # n_jobs=-1: 기본적으로 100개의 결정 트리를 사용하므로 모든 cpu코어 사용
scores=cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)    # return_train_score=True: 훈련 세트에 대한 점수도 반환
print(np.mean(scores['train_score']), np.mean(scores['test_score']))    # 과대적합

0.9973541965122431 0.8905151032797809


In [3]:
# 특성 중요도 출력
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [4]:
# 선택되지 않은 샘플로 훈련한 결정트리 점수 OOB 출력
rf=RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


### 엑스트라 트리 - 랜덤 포레스트와 비슷하지만 노드 분할 시 무작위로 분할

In [ ]:
# 엑스트라 트리 모델
from sklearn.ensemble import ExtraTreesClassifier
et=ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores=cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [6]:
# 모델의 특성 중요도
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


### 그레이디언트 부스팅 - 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완

In [ ]:
# 교차 검증 점수 확인
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(random_state=42)
scores=cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))    # 그레이디언트 부스팅은 과대적합에 강함

0.8881086892152563 0.8720430147331015


In [8]:
# 트리의 개수 증가
gb=GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores=cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [9]:
# 모델의 특성 중요도
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15882696 0.6799705  0.16120254]


### 히스토그램 기반 그레이디언트 부스팅 - 훈련 데이터를 256개의 구간으로 변환하여 분할 속도가 빠름

In [10]:
# 교차 검증 점수
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb=HistGradientBoostingClassifier(random_state=42)
scores=cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

c:\Python310\lib\site-packages\sklearn\experimental\enable_hist_gradient_boosting.py:19: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


0.9321723946453317 0.8801241948619236


In [11]:
# 특성 중요도
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
result=permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [12]:
# 테스트 세트에서 특성 중요도
result=permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [13]:
# 정확도 출력
print(hgb.score(test_input, test_target))

0.8723076923076923


In [15]:
# XGBoost를 사용한 교차 검증 점수
from xgboost import XGBClassifier
xgb=XGBClassifier(tree_method='hist', random_state=42)
scores=cross_validate(xgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9567059184812372 0.8783915747390243


In [17]:
# LightGBM을 사용한 교차 검증 점수
from lightgbm import LGBMClassifier
lgb=LGBMClassifier(random_state=42)
scores=cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
